In [ ]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
#for key,value in datos:
    #print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

In [3]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
    ?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 10000 OFFSET 1880000
"""
datos.load_dataset_from_query(query2, only_uri=False)


#datos.load_dataset_from_nlevels(5, extra_params="LIMIT 100")
datos.show(verbose=False)
dataset1 = datos.train_split()

#     print(sub,dataset1[sub])
datos.save_to_binary("wikidata_2.bin")

# datos.subs

#print(len(jsonlist))
#jsonlist

Exception: Error on endpoint. HTTP status code: 500

In [164]:
import numpy as np
rel = [i[2] for i in datos.subs]
print(len(datos.relations_dict))
print(len(datos.relations))
print(len(datos.entities_dict))
print(len(datos.entities))
rel = np.array(rel)
print(np.sort(rel))
data = np.matrix(datos.subs)
print(data[:,2])

print('Splitting the data into training and validation sets ...')
print(data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
nb_validation_samples = int(0.2 * data.shape[0])


x_train = data[:-nb_validation_samples]
x_val = data[-nb_validation_samples:-int(nb_validation_samples/2)]
x_test = data[-int(nb_validation_samples/2):]

print(x_val.shape,"\n\n\n",x_test.shape,"\n\n\n",x_train.shape)
print(x_test.shape[0]+x_train.shape[0]+x_val.shape[0])


440
440
1497
1497
[  0   0   0 ..., 438 438 439]
[[  0]
 [  1]
 [  3]
 ..., 
 [220]
 [439]
 [ 31]]
Splitting the data into training and validation sets ...
(1727, 3)
(173, 3) 


 (172, 3) 


 (1382, 3)
1727


In [186]:
import importlib
import pickle
import dataset
importlib.reload(dataset)


datos = dataset.Dataset()
datos.load_from_binary("wn18.bin")
datos.show(verbose=False)

40943 entities, 18 relations, 151442 tripletas


In [17]:
import requests

def get_json_offset(n, count):
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
construct {{ ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }}
WHERE {{ ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
}} LIMIT {0} OFFSET {1}
""".format(count, n*count)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    if response.status_code is not 200:
        return False, "Error occurred on http request. Code"+str(response.status_code)
    json_data = response.json()['results']['bindings']
    return json_data

all_json = []
for query in range(0,30):
    json1 = get_json_offset(query,20000)
    all_json = all_json + json1
    


In [6]:
import importlib
import dataset
import queries
import pickle
importlib.reload(dataset)
importlib.reload(queries)

query = queries.Queries()

js = query.big_query(n_levels=3, n_rounds=4, total=10000)
print("finished. Saving")
f1 = open("millon.json", "wb+")
pickle.dump(js, f1)
f1.close()

R0: Offset: 0, limit: 2500
R1: Offset: 2500, limit: 2500
R2: Offset: 5000, limit: 2500
R3: Offset: 7500, limit: 2500
finished. Saving


In [1]:
import importlib
import dataset
import pickle
importlib.reload(dataset)
from datetime import datetime

dtset = dataset.Dataset()


#dataset.load_entire_dataset(1)
dtset.load_dataset_recurrently(1, verbose=2)
dtset.show()



Found 32527 entities
Scanning level 1 with 32663 elements
Enter S to show status: s
Elapsed time: 2s. Depth 1 of 1. Entities scanned: 0.56% (183 of 32663) Active threads: 38
Enter S to show status: s
Elapsed time: 6s. Depth 1 of 1. Entities scanned: 1.59% (519 of 32663) Active threads: 35
Enter S to show status: s
Elapsed time: 9s. Depth 1 of 1. Entities scanned: 2.34% (764 of 32663) Active threads: 37
Enter S to show status: s
Elapsed time: 10s. Depth 1 of 1. Entities scanned: 2.50% (818 of 32663) Active threads: 35
Enter S to show status: s
Elapsed time: 31s. Depth 1 of 1. Entities scanned: 6.18% (2017 of 32663) Active threads: 38
Enter S to show status: s
Elapsed time: 42s. Depth 1 of 1. Entities scanned: 8.06% (2633 of 32663) Active threads: 38
Enter S to show status: s
Elapsed time: 47s. Depth 1 of 1. Entities scanned: 8.93% (2916 of 32663) Active threads: 30
Enter S to show status: s
Elapsed time: 56s. Depth 1 of 1. Entities scanned: 10.39% (3393 of 32663) Active threads: 27
Ente

Found 32472 entities
Scanning level 1 with 32606 elements

Exception in thread Thread-24708:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jovyan/work/dataset.py", line 261, in __all_entity_triplet__
    id_obj = self.add_element(element, self.entities, self.entities_dict)
  File "/home/jovyan/work/dataset.py", line 109, in extract_entity
    if uri[2] == 'www.wikidata.org' and (uri[3] == "reference" and filters['wdt-reference']):
IndexError: list index out of range

Waiting all threads to end
Finalizado
1114242 entities, 1493 relations, 1437504 tripletas
0:09:44.500492



In [2]:
dtset.save_to_binary("wdata1lvl.bin")

True

In [1]:
import importlib
import dataset
import pickle
importlib.reload(dataset)
from datetime import datetime

dataset = dataset.Dataset()


#dataset.load_entire_dataset(1)
dataset.load_from_binary("wikidata_1level.bin")
dataset.show()

1114242 entities, 1493 relations, 1437504 tripletas


In [ ]:
times= [10, 60, 120, 180, 300, 420, 500, 600, 700, 800, 900]
try_catch_all = [548, 2635, None, 7216, 12142, 15622, None, 20500, 23986, 28086, 31591]
if_else_all=[550, 3369, 6771, 10057, 16666, 23575, 28325, None, None, None]
if_else_all2 = [818,3654,6924,17093,None,28298,]